In [2]:

import time
import torch
from torch.distributions import Categorical, kl
from d2l.torch import Animator
from tqdm.autonotebook import tqdm

from net import Net
from aco import ACO
from utils import *


EPS = 1e-10
device = 'cpu'

max_len = {
    100: 4,
    200: 5,
    300: 6,
}

In [3]:
def infer_instance(model, instance, k_sparse, n_ants, t_aco_diff):
    pyg_data, distances, prizes = instance
    if model:
        model.eval()
        heu_mat = model.reshape(pyg_data, model(pyg_data)) + EPS
        aco = ACO(distances, prizes, max_len[len(prizes)], n_ants, heuristic=heu_mat, device=device)
    else:
        aco = ACO(distances, prizes, max_len[len(prizes)], n_ants, device=device, k_sparse=k_sparse)
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost, _ = aco.run(t)
        results[i] = best_cost
    return results
    
@torch.no_grad()
def test(dataset, model, n_ants, k_sparse, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for instance in tqdm(dataset):
        results = infer_instance(model, instance, k_sparse, n_ants, t_aco_diff)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on OP100

MetaACO

In [8]:
n_ants = 20
n_node = 100
k_sparse = 20
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
net = Net().to(device)
net.load_state_dict(torch.load(f'../pretrained/op/op{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net, n_ants, k_sparse, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

  0%|          | 0/100 [00:00<?, ?it/s]

total duration:  1385.5994687080383
T=1, average obj. is 28.945852279663086.
T=10, average obj. is 30.597496032714844.
T=20, average obj. is 30.843215942382812.
T=30, average obj. is 30.915943145751953.
T=40, average obj. is 30.966629028320312.
T=50, average obj. is 31.004030227661133.
T=100, average obj. is 31.07598304748535.


ACO

In [13]:
n_ants = 20
n_node = 100
k_sparse = 20
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
avg_aco_best, duration = test(test_list, None, n_ants, k_sparse, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

  0%|          | 0/100 [00:00<?, ?it/s]

total duration:  1264.759536743164
T=1, average obj. is 16.909893035888672.
T=10, average obj. is 21.29694175720215.
T=20, average obj. is 23.746780395507812.
T=30, average obj. is 25.483396530151367.
T=40, average obj. is 26.736047744750977.
T=50, average obj. is 27.462875366210938.
T=100, average obj. is 29.506956100463867.


### Test on OP200

In [14]:
n_ants = 20
n_node = 200
k_sparse = 50
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
net = Net().to(device)
net.load_state_dict(torch.load(f'../pretrained/op/op{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net, n_ants, k_sparse, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

  0%|          | 0/100 [00:00<?, ?it/s]

total duration:  2543.6038591861725
T=1, average obj. is 47.74967956542969.
T=10, average obj. is 52.0900993347168.
T=20, average obj. is 52.94816970825195.
T=30, average obj. is 53.5031852722168.
T=40, average obj. is 53.72344207763672.
T=50, average obj. is 53.83295440673828.
T=100, average obj. is 54.17605972290039.


In [15]:
n_ants = 20
n_node = 200
k_sparse = 50
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
avg_aco_best, duration = test(test_list, None, n_ants, k_sparse, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

  0%|          | 0/100 [00:00<?, ?it/s]

total duration:  2020.297049999237
T=1, average obj. is 21.552003860473633.
T=10, average obj. is 26.12917709350586.
T=20, average obj. is 31.373525619506836.
T=30, average obj. is 36.82734298706055.
T=40, average obj. is 41.39125061035156.
T=50, average obj. is 44.494476318359375.
T=100, average obj. is 50.48700714111328.
